In [63]:
from utils import pickle_load
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import pandas as pd
import os
from glob import glob
from evaluate import compute_sim, evaluate, l2norm_numpy
from evaluate import dict2list

def l2norm(embs):
    return {k: v/np.linalg.norm(v) for k, v in embs.items()}

infer_dirs = glob('infer/*')[:]
# infer_dirs = infer_dirs[1:]
infer_dirs = [x for x in infer_dirs if 'bk' not in x]
infer_dirs = [x for x in infer_dirs if 'pseudo' in x]
weights = [1.0] * len(infer_dirs)
# weights = [1.0, 0.5, 0.5]

print(f"Ensemble of {len(infer_dirs)} models")

submit_file = f'submission.csv'
if os.path.exists('D:/whale_data/train_images-384-384/train_images-384-384'):
    train_img_dir = 'D:/whale_data/train_images-384-384/train_images-384-384'
else:
    train_img_dir = '/Users/macbook/works/train_images-384-384'

norm=True
method = 'cat'

def get_emb(infer_dirs, subset, weights):
    li = []
    for infer_dir in infer_dirs:
        if isinstance(subset, str):
            embs = pickle_load(f"{infer_dir}/{subset}_emb.pkl")
        else:
            embs = {}
            for s in subset:
                p = f"{infer_dir}/{s}_emb.pkl"
                if os.path.exists(p):
                    embs = {**embs, **pickle_load(p)}
                elif s != "val":
                    raise FileNotFoundError(p)

        li.append(embs)

    if len(li) == 1:
        return li[0]

    print(f'Ensemble from {len(infer_dirs)} models {infer_dirs}')
    li0 = li[0]
    di = {}
    for k in li0.keys():
        di[k] = []
        for i in range(len(li)):
            e = li[i][k]
            if norm:
                e = e / np.linalg.norm(e)
            di[k].append(e)
        
        if method == 'cat':
            di[k] = np.concatenate(di[k], 0)
        else:
            # Mean
            di[k] = np.mean(di[k], 0)

    return di

train_df = pd.read_csv('data/train_kfold.csv')

run_val = False
try:
    train_embs = get_emb(infer_dirs, 'train', weights)
    test_embs = get_emb(infer_dirs, 'test', weights)
    val_embs = get_emb(infer_dirs, 'val', weights)

    train_embs = l2norm(train_embs)
    test_embs = l2norm(test_embs)
    val_embs = l2norm(val_embs)
    run_val = True
    print(len(train_embs) + len(val_embs))
except:
    pass



Ensemble of 5 models
Ensemble from 5 models ['infer\\b5_768_full_m0.4_lb0.815', 'infer\\b5_data15_lb0.822', 'infer\\b5_data20_m0.6_lb0.826', 'infer\\b5_pseudo_lb0.827', 'infer\\v2m_768_lb819']
Ensemble from 5 models ['infer\\b5_768_full_m0.4_lb0.815', 'infer\\b5_data15_lb0.822', 'infer\\b5_data20_m0.6_lb0.826', 'infer\\b5_pseudo_lb0.827', 'infer\\v2m_768_lb819']


In [ ]:
val_df = train_df[train_df.subset == 'test'].reset_index()
val_map = dict(zip(val_df.image, val_df.individual_id))
train_map = dict(zip(train_df.image, train_df.individual_id))
spec_map = dict(zip(train_df.image, train_df.species))
id_map = {**train_map, **val_map}
# val_imgs = val_df.image.unique()
# val_embs = {k: train_embs[k] for k in val_imgs}

In [ ]:
from evaluate import compute_sim, evaluate, map_per_image, compute_simv2
from evaluate import *

if run_val:
    score, val_sim_df = evaluate(train_df, train_embs, val_embs, norm=True)
    val_sim_df["gt"] = val_sim_df.image.map(val_map)
    val_sim_df["map"] = val_sim_df.apply(lambda row: map_per_image(row["gt"], row.predictions.split(" ")), axis=1)
    val_sim_df = val_sim_df.sort_values("map")
    print(score)

In [ ]:
if run_val:
    train_k, train_v = dict2list(train_embs)
    test_k, test_v = dict2list(val_embs)
    class_count_df = train_df.groupby('individual_id').size().to_frame('count').reset_index()
    class_count = dict(zip(class_count_df.individual_id, class_count_df['count']))
    print(train_v.shape, test_v.shape)
    train_v = l2norm_numpy(train_v)
    test_v = l2norm_numpy(test_v)
    train_ids = np.unique([train_map[x] for x in train_k])
    allowed = []
    for i, k in enumerate(test_k):
        if train_map[k] in train_ids:
            allowed.append(i)
    test_k, test_v = [test_k[i] for i in allowed], [test_v[i] for i in allowed]
    cosines = np.matmul(test_v, train_v.T)

In [ ]:
def add_border(img, color):
    bordersize = 14
    return cv2.copyMakeBorder(
        img,
        top=bordersize,
        bottom=bordersize,
        left=bordersize,
        right=bordersize,
        borderType=cv2.BORDER_CONSTANT,
        value=color
    )

if run_val:
    c = 0
    for i, scores in enumerate(cosines):
        if np.random.rand() < 0.1:
            top = 7
            sort_idx = np.argsort(-scores)
            topk = [train_k[j] for j in sort_idx[:top]]
            topk_score = [scores[j] for j in sort_idx[:top]]
            topk_id = [train_map[x] for x in topk]
            qid = test_k[i]
            gt = train_map[qid]
            c+=1
            imgs = [cv2.imread(f'{train_img_dir}/{qid}')[:,:,::-1]]
            for k, l in zip(topk, topk_id):
                im = cv2.imread(f'{train_img_dir}/{k}')[:,:,::-1]
                im = add_border(im, color=(255, 0, 0) if gt != l else (0, 128, 0))
                imgs.append(im)
            # Show image
            fig = plt.figure(figsize=(25, 4))
            columns = top + 1
            rows = 1
            for i2 in range(0, columns*rows):
                fig.add_subplot(rows, columns, i2+1)
                plt.title(f'{gt} {id_map[qid]}' if i2 == 0 else f'{topk_score[i2 - 1]:.2f} {id_map[topk[i2 -1]]}')
                plt.imshow(imgs[i2])
                plt.axis('off')
            
            plt.show()
            
        if c == 4:
            break
        

In [ ]:
# val_sim_df

# sim_df = compute_sim(train_df, {**train_embs, **val_embs}, test_embs, thr=0.5, norm=True)
# sim_df[["image", "predictions"]].to_csv('submission.csv', index=False)
# sim_df.head()   

In [ ]:
# assert False
from sklearn.neighbors import NearestNeighbors
n_neighbors = 100
knn = NearestNeighbors(n_neighbors=n_neighbors,metric='cosine')

# diff fold
db_embs = get_emb(infer_dirs, ['train', 'val'], weights)
test_embs = get_emb(infer_dirs, 'test', weights)

# db_embs = {**train_embs, **val_embs}

train_k, train_v = dict2list(db_embs)
test_k, test_v = dict2list(test_embs)
knn.fit(train_v)

In [ ]:
distances, idxs = knn.kneighbors(test_v, n_neighbors, return_distance=True)

In [ ]:
from tqdm import tqdm

test_df = []
train_k = np.asarray(train_k)
img2id = dict(zip(train_df.image, train_df.individual_id))

for i in tqdm(range(len(test_k))):
    dist, idx = distances[i], idxs[i]
    for d, id in zip(dist, idx):
        img_id = train_k[id]
        tar = img2id[img_id]
        test_df.append([test_k[i], tar, d]) 
    
test_df = pd.DataFrame(test_df, columns=['image', 'target', 'distances'])
test_df['confidence'] = 1-test_df['distances']
test_df = test_df.groupby(['image','target']).confidence.max().reset_index()
test_df = test_df.sort_values('confidence',ascending=False).reset_index(drop=True)
test_df.to_csv('test_neighbors.csv')
test_df.image.value_counts().value_counts()

In [ ]:
predictions = {}
sample_list = ['938b7e931166', '5bf17305f073', '7593d2aee842', '7362d7a01d00','956562ff2888']

for i,row in tqdm(test_df.iterrows(), total=len(test_df)):
    if row.image in predictions:
        if len(predictions[row.image])==5:
            continue
        predictions[row.image].append(row.target)
    elif row.confidence >= 0.5  :
        predictions[row.image] = [row.target,'new_individual']
    else:
        predictions[row.image] = ['new_individual',row.target]

for x in tqdm(predictions):
    if len(predictions[x])<5:
        remaining = [y for y in sample_list if y not in predictions]
        predictions[x] = predictions[x]+remaining
        predictions[x] = predictions[x][:5]
    predictions[x] = ' '.join(predictions[x])
    
predictions = pd.Series(predictions).reset_index()
predictions.columns = ['image','predictions']
predictions.to_csv(submit_file,index=False)
predictions.head()

In [ ]:
assert 0 == 1

In [ ]:
## TEST CV

n_neighbors = 100
knn = NearestNeighbors(n_neighbors=n_neighbors,metric='cosine')

train_k, train_v = dict2list(train_embs)
val_k, val_v = dict2list(val_embs)
knn.fit(train_v)

distances, idxs = knn.kneighbors(val_v, n_neighbors, return_distance=True)

In [ ]:

from tqdm import tqdm

val_df = []
train_k = np.asarray(train_k)
img2id = dict(zip(train_df.image, train_df.individual_id))

for i in tqdm(range(len(val_k))):
    dist, idx = distances[i], idxs[i]
    for d, id in zip(dist, idx):
        img_id = train_k[id]
        tar = img2id[img_id]
        val_df.append([val_k[i], tar, d]) 
    
val_df = pd.DataFrame(val_df, columns=['image', 'target', 'distances'])
val_df['confidence'] = 1-val_df['distances']
val_df = val_df.groupby(['image','target']).confidence.mean().reset_index()
val_df = val_df.sort_values('confidence',ascending=False).reset_index(drop=True)
val_df.to_csv('val_neighbors.csv')
val_df.image.value_counts().value_counts()

val_pred = {}
sample_list = ['938b7e931166', '5bf17305f073', '7593d2aee842', '7362d7a01d00','956562ff2888']

for i,row in tqdm(val_df.iterrows(), total=len(val_df)):
    if row.image in val_pred:
        if len(val_pred[row.image])==5:
            continue
        val_pred[row.image].append(row.target)
    elif row.confidence >= 0.5:
        val_pred[row.image] = [row.target,'new_individual']
    else:
        val_pred[row.image] = ['new_individual',row.target]

for x in tqdm(val_pred):
    if len(val_pred[x])<5:
        remaining = [y for y in sample_list if y not in val_pred]
        val_pred[x] = val_pred[x]+remaining
        val_pred[x] = val_pred[x][:5]
    val_pred[x] = ' '.join(val_pred[x])
    
val_pred = pd.Series(val_pred).reset_index()
val_pred.columns = ['image','predictions']
val_pred.head()

In [ ]:
train_ids = np.unique([train_map[x] for x in train_k])
allowed = []
for i, k in enumerate(val_k):
    if train_map[k] in train_ids:
        allowed.append(train_map[k])

val_targets_df = pd.DataFrame(val_k)
val_targets_df.columns = ['image']
val_targets_df['target'] = val_targets_df['image'].map(train_map)
val_targets_df.loc[~val_targets_df.target.isin(allowed),'target'] = 'new_individual'
val_targets_df.head(10)

all_preds = dict(zip(val_pred['image'], val_pred['predictions']))
th = 0.6
for i,row in val_targets_df.iterrows():
        target = row.target
        preds = all_preds[row.image].split(" ")
        val_targets_df.loc[i,th] = map_per_image(target,preds)
cv = val_targets_df[th].mean()
cv